In [1]:
from mlat.dim2 import TriangularLattice
tr = TriangularLattice(k=[1., 1.], m=[1., 1., 1.], precision=.1)
ws, evecs = tr.dispersion()

# Dispersion relation

In [11]:
import plotly.graph_objects as go


fig = go.Figure(data=[go.Surface(z=ws[:, :, i], x=tr.qxs, y=tr.qys) for i in [4]])
fig.show()

# Animation

In [3]:
import numpy as np
import plotly.graph_objects as go


# Parameters
qx = np.pi / 2
qy = - np.pi / 4
Nx = 5
Ny = 5

idx = min(range(len(tr.qxs)), key=lambda i: abs(tr.qxs[i] - qx))
idy = min(range(len(tr.qys)), key=lambda i: abs(tr.qys[i] - qy))

Q = tr.Q(qx, qy)

mode = 5 # 0 to 5
w = ws[idy, idx, mode] # /s

grid = []
for y in range(Ny):
    x_pos0 = 0
    if y % 2 == 0:
        x_pos0 = 1 / 2
    y_pos = y * np.sqrt(3) / 2
    row = []
    for x in range(Nx):
        row.append([x_pos0 + x, y_pos])
    grid.append(row)
grid = np.array(grid)

_x_axis = grid[:, :, 0]
_y_axis = grid[:, :, 1]


In [4]:
# Print infos
print("Angular frequency: ", w)
print("Wave number: \nqx: ", qx, "\nqy: ", qy)


def generate_dgrid(shape, evecs_A, evecs_B, evecs_C):
    dgrid = np.zeros(shape, dtype=np.complex128)

    # Process first two rows
    # CA...
    # BC...
    dgrid[1, 1] = evecs_A
    dgrid[0, 0] = evecs_B
    dgrid[0, 1] = evecs_C
    dgrid[1, 0] = evecs_C * Q[1].conj()

    for x in range(2, Nx):
        dgrid[0, x] = dgrid[1, x - 1] * Q[1]
        dgrid[1, x] = dgrid[0, x - 2] * Q[0]

    for y in range(2, Ny):
        for x in range(Nx):
            dgrid[y, x] = dgrid[y - 2, x] * Q[2]

    return dgrid

evecs_A = evecs[idy, idx, mode, :2]
evecs_B = evecs[idy, idx, mode, 2:4]
evecs_C = evecs[idy, idx, mode, 4:]
dgrid = generate_dgrid(grid.shape, evecs_A, evecs_B, evecs_C)

Angular frequency:  2.3031911249328507
Wave number: 
qx:  1.5707963267948966 
qy:  -0.7853981633974483


In [5]:
# Parameters for animation
fps = 30 # /s
animation_length = 3 # s

# Construct frames
frames = []
for t in range(int(animation_length * fps)):
    dt = t / fps
    dphase = dt * w * 2 * np.pi
    current_grid = grid + dgrid * np.exp(1.j * dphase)
    x = current_grid[:, :, 0].flatten()
    y = current_grid[:, :, 1].flatten()
    frames.append(go.Frame(data=[go.Scatter(x=x.real, y=y.real, mode='markers')]))

# Figure components
start_button = dict(
        label="Play",
        method="animate",
        args=[
            None, 
            {
                "frame":{"duration": 1000 / fps, "redraw": False},
                "fromcurrent": True, 
                "transition": {"duration": 100}
            }])
pause_button = dict(
        label="Pause",
        method="animate",
        args=[
            [None], 
            {
                "frame": {"duration": 0, "redraw": False},
                "mode": "immediate",
                "transition": {"duration": 0}
            }])

# Plot
fig = go.Figure(
    data=frames[0].data,
    layout=go.Layout(
        title="Dispersion relation animation",
        yaxis=dict(scaleanchor="x", scaleratio=1, autorange='reversed'),
        updatemenus=[
            dict(
                type="buttons",
                buttons=[start_button, pause_button
            ])
        ]
    ),
    frames=frames[1:])
fig.show()